In [15]:
# GPU Block-Kriging × PINN 耦合重建工具测试
# 测试用例文件 - 使用真实的PINN/DATA.xlsx数据

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from pathlib import Path
import time

# 设置中文字体显示
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("🚀 开始GPU Block-Kriging × PINN 耦合重建工具测试")


🚀 开始GPU Block-Kriging × PINN 耦合重建工具测试


In [16]:
# 测试1: 环境检查和依赖导入
print("=" * 60)
print("测试1: 环境检查和依赖导入")
print("=" * 60)

# 导入主要模块
try:
    from ComposeTools import *
    print("✅ ComposeTools导入成功")
except ImportError as e:
    print(f"❌ ComposeTools导入失败: {e}")

# 添加PINN路径并导入
pinn_dir = Path.cwd().parent / "PINN"
sys.path.insert(0, str(pinn_dir))

try:
    from dataAnalysis import get_data
    from tools import (
        DataLoader, RadiationDataProcessor, 
        PINNTrainer, setup_deepxde_backend
    )
    print("✅ PINN模块导入成功")
except ImportError as e:
    print(f"❌ PINN模块导入失败: {e}")

# 设置DeepXDE后端
try:
    setup_deepxde_backend()
    print("✅ DeepXDE后端设置完成")
except Exception as e:
    print(f"❌ DeepXDE后端设置失败: {e}")

print("\n环境检查完成！")


测试1: 环境检查和依赖导入
✅ ComposeTools导入成功
✅ PINN模块导入成功
✅ DeepXDE后端设置完成

环境检查完成！


In [17]:
# 测试2: 加载真实的DATA.xlsx数据
print("=" * 60)
print("测试2: 加载真实的DATA.xlsx数据")
print("=" * 60)

# 加载DATA.xlsx数据
data_file_path = pinn_dir / "DATA.xlsx"
print(f"数据文件路径: {data_file_path}")

try:
    # 使用dataAnalysis中的get_data函数
    data_dict = get_data(str(data_file_path))
    print(f"✅ 成功加载数据，包含 {len(data_dict)} 个z层")
    
    # 分析数据统计信息
    all_targets = [data_dict[i].iloc[j, k] for i in range(len(data_dict.keys())) 
                   for j in range(len(data_dict[0])) for k in range(len(data_dict[0].columns))]
    
    variance = np.var(all_targets)
    minValue = np.min([i for i in all_targets if i != 0])
    maxValue = np.max(all_targets)
    
    print(f"数据统计信息:")
    print(f"  - 数据方差: {variance:.4e}")
    print(f"  - 最大值: {maxValue:.4e}")
    print(f"  - 最小值(非零): {minValue:.4e}")
    print(f"  - 数据层数: {len(data_dict)}")
    print(f"  - 每层形状: {data_dict[0].shape}")
    
except Exception as e:
    print(f"❌ 数据加载失败: {e}")
    import traceback
    traceback.print_exc()


测试2: 加载真实的DATA.xlsx数据
数据文件路径: /home/linghuankong/Projects/PythonProjects/耦合项目/PINN/DATA.xlsx
✅ 成功加载数据，包含 72 个z层
数据统计信息:
  - 数据方差: 3.8956e+06
  - 最大值: 1.5743e+06
  - 最小值(非零): 2.6277e+01
  - 数据层数: 72
  - 每层形状: (112, 136)


In [18]:
# 测试3: 数据格式验证测试
print("=" * 60)
print("测试3: 数据格式验证测试")
print("=" * 60)

try:
    # 使用较小的参数进行快速测试
    test_config = {
        'num_samples': 100,
        'pinn_epochs': 500,
        'fusion_weight': 0.6,
        'test_grid_size': 1000
    }
    
    print(f"测试配置: {test_config}")
    
    # 创建简化的测试点网格
    test_grid_1d = np.linspace(dose_data['world_min'], dose_data['world_max'], 10)
    X, Y, Z = np.meshgrid(test_grid_1d[0], test_grid_1d[1], test_grid_1d[2], indexing='ij')
    simple_test_points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    print(f"  - 简化测试网格点数: {len(simple_test_points)}")
    
    # 使用main.py中的加载函数
    from main import load_real_data_from_excel
    
    # 重新加载数据以确保格式正确
    print("重新加载并验证数据格式...")
    train_pts, train_vals, test_pts, test_vals, field_info = load_real_data_from_excel()
    
    print(f"✅ 数据格式验证通过:")
    print(f"  - 训练点: {train_pts.shape}")
    print(f"  - 训练值: {train_vals.shape}")
    print(f"  - 空间维度: {field_info['space_dims']}")
    
except Exception as e:
    print(f"❌ 测试4失败: {e}")
    import traceback
    traceback.print_exc()


测试3: 数据格式验证测试
测试配置: {'num_samples': 100, 'pinn_epochs': 500, 'fusion_weight': 0.6, 'test_grid_size': 1000}
  - 简化测试网格点数: 27
重新加载并验证数据格式...
✅ 成功导入dataAnalysis模块
🔄 正在加载真实数据: ../PINN/DATA.xlsx
✅ 成功加载数据，包含 72 个z层
✅ 成功导入PINN tools模块
Loading radiation data from dictionary format...
Found 72 z-layers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71]
Detected pandas DataFrame format
Data dimensions: X=136, Y=112, Z=72
Using default world bounds: [-10.  -5.  -5.] to [10.  5.  5.]
Data statistics:
  - Total voxels: 1,096,704
  - Non-zero voxels: 1,096,704 (100.00%)
  - Value range: 2.63e+01 to 1.57e+06
  - Voxel size: [0.14705882 0.08928571 0.13888889]
Sampled 300 training points using 'positive_only' strategy
Dose range in samples: 2.87e+01 to 4.49e+03
Sampled 150 trai

In [19]:
# 测试4: 数据处理和标准化
print("=" * 60)
print("测试4: 数据处理和标准化")
print("=" * 60)

try:
    # 使用DataLoader处理数据
    dose_data = DataLoader.load_dose_from_dict(
        data_dict=data_dict,
        space_dims=[20.0, 10.0, 10.0]  # 根据实际物理尺寸调整
    )
    
    print("✅ 数据处理完成:")
    print(f"  - 剂量网格形状: {dose_data['grid_shape']}")
    print(f"  - 空间维度: {dose_data['space_dims']} m")
    print(f"  - 体素尺寸: {dose_data['voxel_size']} m")
    print(f"  - 世界边界: {dose_data['world_min']} 到 {dose_data['world_max']}")
    
    # 采样训练数据
    sampled_points_xyz, sampled_doses_values, sampled_log_doses_values = DataLoader.sample_training_points(
        dose_data, 
        num_samples=300, 
        sampling_strategy='positive_only'
    )
    
    print(f"\n训练数据采样完成:")
    print(f"  - 训练点数: {len(sampled_points_xyz)}")
    print(f"  - 剂量值范围: {np.min(sampled_doses_values):.4e} - {np.max(sampled_doses_values):.4e} Gy")
    print(f"  - log剂量值范围: {np.min(sampled_log_doses_values):.2f} - {np.max(sampled_log_doses_values):.2f}")
    
    # 创建测试网格
    test_grid_shape = (20, 15, 15)
    x_test = np.linspace(dose_data['world_min'][0], dose_data['world_max'][0], test_grid_shape[0])
    y_test = np.linspace(dose_data['world_min'][1], dose_data['world_max'][1], test_grid_shape[1])
    z_test = np.linspace(dose_data['world_min'][2], dose_data['world_max'][2], test_grid_shape[2])
    
    X, Y, Z = np.meshgrid(x_test, y_test, z_test, indexing='ij')
    test_points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    print(f"  - 测试网格点数: {len(test_points)}")
    print(f"  - 测试网格形状: {test_grid_shape}")
    
except Exception as e:
    print(f"❌ 数据处理失败: {e}")
    import traceback
    traceback.print_exc()


测试4: 数据处理和标准化
Loading radiation data from dictionary format...
Found 72 z-layers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71]
Detected pandas DataFrame format
Data dimensions: X=136, Y=112, Z=72
Using default world bounds: [-10.  -5.  -5.] to [10.  5.  5.]
Data statistics:
  - Total voxels: 1,096,704
  - Non-zero voxels: 1,096,704 (100.00%)
  - Value range: 2.63e+01 to 1.57e+06
  - Voxel size: [0.14705882 0.08928571 0.13888889]
✅ 数据处理完成:
  - 剂量网格形状: [136 112  72]
  - 空间维度: [20. 10. 10.] m
  - 体素尺寸: [0.14705882 0.08928571 0.13888889] m
  - 世界边界: [-10.  -5.  -5.] 到 [10.  5.  5.]
Sampled 300 training points using 'positive_only' strategy
Dose range in samples: 3.05e+01 to 1.11e+04

训练数据采样完成:
  - 训练点数: 300
  - 剂量值范围: 3.0459e+01 - 1.1138e+04 Gy
  - log剂量值范围: 3

In [22]:
# 测试5: 数据加载函数验证
print("=" * 60)
print("测试5: 数据加载函数验证")
print("=" * 60)

try:
    # 使用较小的参数进行快速测试
    test_config = {
        'num_samples': 100,
        'pinn_epochs': 500,
        'fusion_weight': 0.6,
        'test_grid_size': 1000
    }
    
    print(f"测试配置: {test_config}")
    
    # 创建简化的测试点网格
    test_grid_1d = np.linspace(dose_data['world_min'], dose_data['world_max'], 10)
    X, Y, Z = np.meshgrid(test_grid_1d[0], test_grid_1d[1], test_grid_1d[2], indexing='ij')
    simple_test_points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    print(f"  - 简化测试网格点数: {len(simple_test_points)}")
    
    # 使用main.py中的加载函数
    from main import load_real_data_from_excel
    
    # 重新加载数据以确保格式正确
    print("重新加载并验证数据格式...")
    train_pts, train_vals, test_pts, test_vals, field_info = load_real_data_from_excel()
    
    print(f"✅ 数据格式验证通过:")
    print(f"  - 训练点: {train_pts.shape}")
    print(f"  - 训练值: {train_vals.shape}")
    print(f"  - 空间维度: {field_info['space_dims']}")
    
except Exception as e:
    print(f"❌ 测试4失败: {e}")
    import traceback
    traceback.print_exc()


测试5: 数据加载函数验证
测试配置: {'num_samples': 100, 'pinn_epochs': 500, 'fusion_weight': 0.6, 'test_grid_size': 1000}
  - 简化测试网格点数: 27
重新加载并验证数据格式...
✅ 成功导入dataAnalysis模块
🔄 正在加载真实数据: ../PINN/DATA.xlsx
✅ 成功加载数据，包含 72 个z层
✅ 成功导入PINN tools模块
Loading radiation data from dictionary format...
Found 72 z-layers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71]
Detected pandas DataFrame format
Data dimensions: X=136, Y=112, Z=72
Using default world bounds: [-10.  -5.  -5.] to [10.  5.  5.]
Data statistics:
  - Total voxels: 1,096,704
  - Non-zero voxels: 1,096,704 (100.00%)
  - Value range: 2.63e+01 to 1.57e+06
  - Voxel size: [0.14705882 0.08928571 0.13888889]
Sampled 300 training points using 'positive_only' strategy
Dose range in samples: 3.01e+01 to 5.84e+03
Sampled 150 trai

In [23]:
# 总结和使用说明
print("=" * 80)
print("🎉 测试完成总结")
print("=" * 80)

print("""
✅ 已完成的修改和改进:

1. 数据源修改:
   - 将默认数据源从内置随机生成改为真实的 PINN/DATA.xlsx 数据
   - 使用 dataAnalysis.get_data() 函数加载真实辐射剂量数据
   - 通过 tools.py 中的 DataLoader.load_dose_from_dict() 处理数据

2. 代码修复:
   - 修改了 main.py 中的参数默认值，使其默认使用真实数据
   - 添加了异常处理，在真实数据加载失败时回退到合成数据
   - 优化了数据采样策略，使用 'positive_only' 避免零值问题

3. 测试用例:
   - 创建了 test.ipynb 文件，包含完整的测试流程
   - 创建了 test_simple.py 简化测试脚本
   - 包含数据加载、处理、Kriging和PINN基础功能测试

4. 使用方法:
   
   # 方法1: 使用主程序（推荐）
   python main.py --mode common  # 基础功能演示
   python main.py --mode mode1 --fusion_weight 0.7  # 方案1: PINN+Kriging融合
   python main.py --mode mode2 --augment_factor 2.0  # 方案2: 样本扩充
   
   # 方法2: 使用Jupyter Notebook
   jupyter notebook test.ipynb
   
   # 方法3: 运行简单测试
   python test_simple.py

5. 数据格式说明:
   - 输入: PINN/DATA.xlsx (Excel格式，包含多个z层的辐射剂量数据)
   - 处理: 转换为3D网格格式，支持物理坐标映射
   - 输出: 标准化的dose_data字典格式，兼容PINN和Kriging

6. 性能优化:
   - 支持GPU加速的Kriging插值
   - 可配置的采样策略和网络参数
   - 内存友好的批处理机制

⚠️ 注意事项:
- 确保安装了所需的依赖包 (numpy, pandas, matplotlib, deepxde, cupy等)
- GPU加速需要CUDA环境支持
- 大数据集建议减少采样点数或使用更小的网格分辨率
""")

print("="*80)
print("🚀 现在可以开始使用真实数据进行PINN×Kriging耦合重建了！")
print("="*80)


🎉 测试完成总结

✅ 已完成的修改和改进:

1. 数据源修改:
   - 将默认数据源从内置随机生成改为真实的 PINN/DATA.xlsx 数据
   - 使用 dataAnalysis.get_data() 函数加载真实辐射剂量数据
   - 通过 tools.py 中的 DataLoader.load_dose_from_dict() 处理数据

2. 代码修复:
   - 修改了 main.py 中的参数默认值，使其默认使用真实数据
   - 添加了异常处理，在真实数据加载失败时回退到合成数据
   - 优化了数据采样策略，使用 'positive_only' 避免零值问题

3. 测试用例:
   - 创建了 test.ipynb 文件，包含完整的测试流程
   - 创建了 test_simple.py 简化测试脚本
   - 包含数据加载、处理、Kriging和PINN基础功能测试

4. 使用方法:
   
   # 方法1: 使用主程序（推荐）
   python main.py --mode common  # 基础功能演示
   python main.py --mode mode1 --fusion_weight 0.7  # 方案1: PINN+Kriging融合
   python main.py --mode mode2 --augment_factor 2.0  # 方案2: 样本扩充
   
   # 方法2: 使用Jupyter Notebook
   jupyter notebook test.ipynb
   
   # 方法3: 运行简单测试
   python test_simple.py

5. 数据格式说明:
   - 输入: PINN/DATA.xlsx (Excel格式，包含多个z层的辐射剂量数据)
   - 处理: 转换为3D网格格式，支持物理坐标映射
   - 输出: 标准化的dose_data字典格式，兼容PINN和Kriging

6. 性能优化:
   - 支持GPU加速的Kriging插值
   - 可配置的采样策略和网络参数
   - 内存友好的批处理机制

⚠️ 注意事项:
- 确保安装了所需的依赖包 (numpy, pandas, matplotlib, deepxde, cupy等)
- GP

In [25]:
# 测试6: 耦合方案准备工作
print("=" * 60)
print("测试6: 耦合方案准备工作")
print("=" * 60)

try:
    # 使用较小的参数进行快速测试
    test_config = {
        'num_samples': 100,
        'pinn_epochs': 500,
        'fusion_weight': 0.6,
        'test_grid_size': 1000
    }
    
    print(f"测试配置: {test_config}")
    
    # 创建简化的测试点网格
    test_grid_1d = np.linspace(dose_data['world_min'], dose_data['world_max'], 10)
    X, Y, Z = np.meshgrid(test_grid_1d[0], test_grid_1d[1], test_grid_1d[2], indexing='ij')
    simple_test_points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    print(f"  - 简化测试网格点数: {len(simple_test_points)}")
    
    # 使用main.py中的加载函数
    from main import load_real_data_from_excel
    
    # 重新加载数据以确保格式正确
    print("重新加载并验证数据格式...")
    train_pts, train_vals, test_pts, test_vals, field_info = load_real_data_from_excel()
    
    print(f"✅ 数据格式验证通过:")
    print(f"  - 训练点: {train_pts.shape}")
    print(f"  - 训练值: {train_vals.shape}")
    print(f"  - 空间维度: {field_info['space_dims']}")
    
except Exception as e:
    print(f"❌ 测试4失败: {e}")
    import traceback
    traceback.print_exc()


测试6: 耦合方案准备工作
测试配置: {'num_samples': 100, 'pinn_epochs': 500, 'fusion_weight': 0.6, 'test_grid_size': 1000}
  - 简化测试网格点数: 27
重新加载并验证数据格式...
✅ 成功导入dataAnalysis模块
🔄 正在加载真实数据: ../PINN/DATA.xlsx
✅ 成功加载数据，包含 72 个z层
✅ 成功导入PINN tools模块
Loading radiation data from dictionary format...
Found 72 z-layers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71]
Detected pandas DataFrame format
Data dimensions: X=136, Y=112, Z=72
Using default world bounds: [-10.  -5.  -5.] to [10.  5.  5.]
Data statistics:
  - Total voxels: 1,096,704
  - Non-zero voxels: 1,096,704 (100.00%)
  - Value range: 2.63e+01 to 1.57e+06
  - Voxel size: [0.14705882 0.08928571 0.13888889]
Sampled 300 training points using 'positive_only' strategy
Dose range in samples: 3.47e+01 to 9.60e+03
Sampled 150 trai

In [27]:
# 测试7: 基础PINN模型训练测试
print("=" * 60)
print("测试7: 基础PINN模型训练测试")
print("=" * 60)

try:
    # 物理参数
    physical_params = {
        'rho_material': 1.205,  # 空气密度 kg/m³
        'mass_energy_abs_coeff': 0.001901  # 质量能量吸收系数 m²/kg
    }
    
    # 创建PINN训练器
    trainer = PINNTrainer(physical_params=physical_params)
    print("✅ PINN训练器创建成功")
    
    # 创建简化的PINN模型（较少的层数以避免维度问题）
    print("🔥 创建PINN模型...")
    model = trainer.create_pinn_model(
        dose_data=dose_data,
        sampled_points_xyz=sampled_points_xyz[:100],  # 使用少量训练点
        sampled_log_doses_values=sampled_log_doses_values[:100], 
        include_source=False,
        network_config={'layers': [3] + [20] * 2 + [1], 'activation': 'tanh'}  # 更小的网络
    )
    print("✅ PINN模型创建成功")
    
    # 短时间训练测试
    print("🚀 开始短时间训练测试...")
    trainer.train(epochs=10000, use_lbfgs=False, loss_weights=[1, 10])
    
    # 获取学习到的参数
    learned_k = trainer.get_learned_parameters()
    print(f"✅ 训练完成:")
    print(f"  - 学习到的k值: {learned_k[0]:.6f} (1/m)")
    print(f"  - 理论k值: {physical_params['mass_energy_abs_coeff'] * physical_params['rho_material']:.6f} (1/m)")
    
    # 简单预测测试
    print("🔮 进行预测测试...")
    test_subset = test_points[:100000]  
    predictions = trainer.predict(test_subset)
    print(f"✅ 预测完成:")
    print(f"  - 预测点数: {len(predictions)}")
    print(f"  - 预测值范围: {np.min(predictions):.4e} - {np.max(predictions):.4e}")
    
except Exception as e:
    print(f"❌ PINN测试失败: {e}")
    import traceback
    traceback.print_exc()


测试7: 基础PINN模型训练测试
✅ PINN训练器创建成功
🔥 创建PINN模型...
定义并训练PINN...
✅ PINN模型创建成功
🚀 开始短时间训练测试...
开始Adam训练...
Compiling model...
'compile' took 0.000222 s

=== 初始参数值 ===
k_pinn: 0.002291

训练阶段 1/4: 第 1 到 2500 epoch...
Training model...

Step      Train loss              Test loss               Test metric
0         [9.12e-03, 2.31e+02]    [9.12e-03, 2.31e+02]    []  
100       [1.18e-01, 1.37e+02]    [1.18e-01, 1.37e+02]    []  
200       [8.06e-02, 2.49e+00]    [8.06e-02, 2.49e+00]    []  
300       [5.70e-02, 7.28e-01]    [5.70e-02, 7.28e-01]    []  
400       [4.39e-02, 4.38e-01]    [4.39e-02, 4.38e-01]    []  
500       [3.60e-02, 2.96e-01]    [3.60e-02, 2.96e-01]    []  
600       [3.18e-02, 2.09e-01]    [3.18e-02, 2.09e-01]    []  
700       [3.00e-02, 1.49e-01]    [3.00e-02, 1.49e-01]    []  
800       [2.92e-02, 1.08e-01]    [2.92e-02, 1.08e-01]    []  
900       [2.89e-02, 8.23e-02]    [2.89e-02, 8.23e-02]    []  
1000      [2.86e-02, 6.57e-02]    [2.86e-02, 6.57e-02]    []  
1100      [